### Forecasting of electricity consumption using a simple neural network

In [26]:
import torch
import pandas as pd
import numpy as np

In [27]:
# split a univariate sequence into samples
def split_sequenceUStep(sequence, n_steps_in):
    X, y = list(), list()
    
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

In [28]:
# Load the data
data = pd.read_csv('electricity_demand.csv', header=0, index_col=0)

In [29]:
# Prepare the data
# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
train_data = data.iloc[:train_size]
test_data  = data.iloc[train_size:]

In [30]:
x1,y1 = split_sequenceUStep(train_data.values, 168)
x2,y2 = split_sequenceUStep(test_data.values , 168)

In [31]:
# Convert the data into PyTorch tensors
train_inputs = torch.from_numpy(x1)
train_targets = torch.from_numpy(y1)
test_inputs = torch.from_numpy(x2)
test_targets = torch.from_numpy(y2)

In [32]:
# # Convert the data into PyTorch tensors
# train_inputs = torch.from_numpy(train_data.values[:-1, :])
# train_targets = torch.from_numpy(train_data.values[1:, :])
# test_inputs = torch.from_numpy(test_data.values[:-1, :])
# test_targets = torch.from_numpy(test_data.values[1:, :])

In [33]:
# Define the neural network model
class ElectricityDemandForecasting(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_layer1 = torch.nn.Linear(input_size, hidden_size)
        self.hidden_layer2 = torch.nn.Linear(hidden_size, hidden_size)
        self.output_layer = torch.nn.Linear(hidden_size, output_size)

    def forward(self, input):
        hidden = torch.relu(self.hidden_layer1(input))
        hidden = torch.relu(self.hidden_layer2(hidden))
        output = self.output_layer(hidden)
        return output

In [36]:
# Initialize the model and optimizer
model = ElectricityDemandForecasting(input_size=1, hidden_size=200, output_size=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [39]:
# Train the model
for epoch in range(100):
    # Forward pass
    train_predictions = model(train_inputs.float())
    train_loss = torch.nn.MSELoss()(train_predictions, train_inputs.float())

    # Backward pass
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    # Print the training loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}, Train Loss: {train_loss.item()}')

Epoch 10, Train Loss: 0.017689518630504608
Epoch 20, Train Loss: 0.002716906601563096
Epoch 30, Train Loss: 0.0003838106931652874
Epoch 40, Train Loss: 0.0008397329365834594
Epoch 50, Train Loss: 0.00036832658224739134
Epoch 60, Train Loss: 6.422441219910979e-05


In [ ]:
# Evaluate the model on the test set
test_predictions = model(test_inputs.float())
test_loss = torch.nn.MSELoss()(test_predictions, test_inputs.float())
print(f'Test Loss: {test_loss.item()}')

Test Loss: 0.0008361738291569054


In [ ]:
# Save the trained model
torch.save(model.state_dict(), "model.pth")

In [ ]:
# # Load the saved model
# model.load_state_dict(torch.load("model.pth"))
# model.eval()

# # Predict demand for new input
# with torch.no_grad():
#     predicted_demand = model(new_input)


In [ ]:
# Convert the test predictions and test output to NumPy arrays
test_predictions = test_predictions.detach().numpy()


In [ ]:
test_predictions_df = pd.DataFrame(test_predictions, columns=['Predicted Demand'])
test_predictions_df.to_csv('test_predictions.csv', index=False)

In [ ]:
# # import pandas as pd
# import altair as alt

# # # Convert the test predictions and test output to Pandas dataframes
# # test_predictions_df = pd.DataFrame(test_predictions, columns=['Predicted Demand'])
# # test_output_df = pd.DataFrame(test_output, columns=['Actual Demand'])

# # Concatenate the two dataframes
# results_df = pd.concat([test_targets_df, test_predictions_df], axis=0)

# # Create an Altair chart
# chart = alt.Chart(results_df).mark_line().encode(
#     # x='index:Q',
#     y=alt.Y('value:Q', axis=alt.Axis(title='Electricity Demand (kW)')),
#     color='variable:N'
# ).properties(
#     title='Electricity Demand Forecasting Results',
#     width=600,
#     height=400
# ).transform_fold(
#     fold=['Actual Demand', 'Predicted Demand'],
#     as_=['variable', 'value']
# ).interactive()

# # Display the chart
# chart